In [1]:
import pandas as pd
import nasdaqdatalink as nddl
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import numpy as np
import os
import hvplot.pandas

# store the api key (just the key in a text file with this name)
nddl.read_key(filename="nasdaqdatalinkapikey.txt")


In [2]:
# get the indicator table
zindicators = nddl.get_table("ZILLOW/INDICATORS")
zindicators

ForbiddenError: (Status 403) (Nasdaq Data Link Error QEPx04) A valid API key is required to retrieve data. Please check your API key and try again. You can find your API key under your account settings.

In [ ]:
# retrieve smooth median house price data
# get the data table by indicator (the whole table is huge), paginate= True will retrieve more than 10K rows, otherwise limit to 10K rows.
sssm_data = nddl.get_table("ZILLOW/DATA", indicator_id='SSSM', paginate=True)
sssm_data

In [ ]:
# get SRSM, raw median price data
srsm_data = nddl.get_table("ZILLOW/DATA", indicator_id='SRSM', paginate=True)
srsm_data

In [ ]:
# For-Sale Inventory (Smooth, SFR only, Monthly) ISSM
issm = nddl.get_table("ZILLOW/DATA", indicator_id='ISSM', paginate=True)
issm

In [ ]:
# retrieve raw inventory data
irsm = nddl.get_table("ZILLOW/DATA", indicator_id='IRSM', paginate=True)
irsm

In [ ]:
# SRAM, Median Sale Price (Raw, All Homes, Monthly), meaning all house types, previously SSSM is for SFR (single family residence)
sram = nddl.get_table("ZILLOW/DATA", indicator_id='SRAM', paginate=True)
sram

In [ ]:
# get all the regions table
zregions = nddl.get_table("ZILLOW/REGIONS", paginate=True)
zregions

In [ ]:
zregions['region_id'] = zregions['region_id'].astype('int64')

In [ ]:
zregions.dtypes

In [ ]:
# combine data and region df
sssm_combine = pd.merge(sssm_data, zregions)
sssm_combine

In [ ]:
# combine data and region df raw
srsm_combine = pd.merge(srsm_data, zregions)
srsm_combine

In [ ]:
sram_combine = pd.merge(sram, zregions)
sram_combine.head()

In [ ]:
sram_combine[['City', 'State']] = sram_combine['region'].str.split(', ', expand=True)
sram_combine.State.unique()

In [ ]:
# create new columns City and State
srsm_combine[['City', 'State']]= srsm_combine['region'].str.split(', ', expand=True)
srsm_combine

In [ ]:
srsm_combine.State.unique()

In [ ]:
# data cleaning
# replace (probably input error) "NC; NC" with "NC"
srsm_combine['State'] = srsm_combine['State'].str.replace("NC; NC", "NC")

In [ ]:
sram_combine['State'] = sram_combine['State'].str.replace("NC; NC", "NC")
sram_combine.State.unique()

In [ ]:
sssm_combine[['City', 'State']]= sssm_combine['region'].str.split(', ', expand=True)
# sssm_combine['State'] = sssm_combine['State'].str.strip()
sssm_combine

In [ ]:
# replace (probably input error) "NC; NC" with "NC"
sssm_combine['State'] = sssm_combine['State'].str.replace("NC; NC", "NC")

In [ ]:
sssm_combine.State.unique()

In [ ]:
sssm_combine[sssm_combine['State'].isin([None, 'NC; NC'])]


In [ ]:
sssm_combine.describe(include='all')

In [ ]:
srsm_combine.describe(include='all')

# Do NOT run when combining code if the CSV files already exist

In [ ]:

# create directory to store data (csv files)
dirpath = ('./data')
if not os.path.exists(dirpath):
    os.makedirs(dirpath)
    print('created ./data/ directory')
else:
    print('./data directory already exist')

In [ ]:
# save pd to csv file for use next time without calling API again
zindicators.to_csv('./data/zindicators.csv')
zregions.to_csv('./data/zillow_region.csv')

sssm_data.to_csv('./data/sssm_full.csv')
srsm_data.to_csv('./data/srsm_full.csv')

sssm_combine.to_csv('./data/sssm_combine.csv')
srsm_combine.to_csv('./data/srsm_combine.csv')
sram_combine.to_csv('./data/sram_combine.csv')

irsm.to_csv('./data/irsm_full.csv')
issm.to_csv('./data/issm.csv')


## Findings about sssm_combine  
  - It contains smooth metropolitan area monthly median house price over the years ( in each state as well as the whole of US (region = 'United States'), 90705 rows x 8 columns.  
  - The date range may defer from city to city.   
  - Different states have different numbers of metro area (AK, Alaska, only has 1, CA, california, has 34.  
  - The difference between smoothed and raw data on Zillow is that the former removes 'spikes' in the raw data to make the chart look 'smoother', which is fine for our purpose. Data wise, the smoothed data will have their tail components adjusted to give the graph a "smoother" appearance, otherwise the raw data may have many "steps" or small flat lines. (e.g 130,750 (smooth) vs 130,000 (raw)). According to Zillow, the adjustment may also be for seasonal reason.  
      - The raw data also have slightly more data points than the smoothed data (93075 vs 92558, 0.56% less).  
      - Raw data also has State typo (eg. "NC; NC"), which may be introduced during data input.
### To do:  
  1. -find the city with the longest data period (groupby region, count)    (Dayton, 357 pt)
  2. -plot that city with the national data  
  3. plot the cities in CA.  
  4. get the number of cities in each state.

In [ ]:
sssm_combine = pd.read_csv('./data/sssm_combine.csv', parse_dates=['date'])
# drop the first col (index number)
sssm_combine = sssm_combine.drop(sssm_combine.columns[0], axis=1)
sssm_combine

In [ ]:
srsm_combine = pd.read_csv('./data/srsm_combine.csv', parse_dates=['date'])
# drop the first col (index number)
srsm_combine = srsm_combine.drop(srsm_combine.columns[0], axis=1)
srsm_combine

In [ ]:
srsm_combine.dtypes

In [ ]:
issm =pd.read_csv('./data/issm.csv', parse_dates=['date'])
issm= issm.drop(issm.columns[0], axis=1)
issm.dtypes

In [ ]:
issm.head()

In [ ]:
irsm =pd.read_csv('./data/irsm.csv', parse_dates=['date'])
irsm= irsm.drop(irsm.columns[0], axis=1)
irsm.dtypes

In [ ]:
sssm_combine.dtypes

In [ ]:
# extract national data in sssm_combine
usa_sssm_comb = sssm_combine[sssm_combine['State'].isnull() ]
usa_sssm_comb

In [ ]:
# extract national data in srsm_combine
usa_srsm_comb = srsm_combine[srsm_combine['State'].isnull() ]
usa_srsm_comb

In [ ]:
# remove national data from sssm_combine
sssm_combine = sssm_combine.dropna()
sssm_combine

In [ ]:
# remove country-wide data from srsm_combine
srsm_combine = srsm_combine.dropna()
srsm_combine.tail()

In [ ]:
sssm_combine.groupby('region')['State'].count().sort_values()

In [ ]:
srsm_combine.groupby('region')['State'].count().sort_values()

In [ ]:
# extract Dayton, OH smooth data points
dayton = sssm_combine[sssm_combine['region']== 'Dayton, OH']
dayton_39 = dayton[dayton['region_id'] == 394521]
dayton_39.sort_values(by='date').tail()

In [ ]:
# extract Dayton, OH Raw data points
dayton_r = srsm_combine[srsm_combine['region']== 'Dayton, OH']
# dayton_r.region_id.unique()
dayton_39r = dayton_r[dayton_r['region_id'] == 394521]
dayton_39r.sort_values(by='date').tail()

In [ ]:
# extract new york raw data points
ny_r = srsm_combine[srsm_combine['region']== 'New York, NY']
ny_r.head()

In [ ]:
# extract Killeen raw data points
killeen_r = srsm_combine[srsm_combine['region']== 'Killeen, TX']
len(killeen_r)
killeen_r.region_id.unique()
killeen_r

In [ ]:
# extract Davenport raw data points
davenport_r = srsm_combine[srsm_combine['region'] == 'Davenport, IA']
davenport_r.region_id.unique()

In [ ]:
# plotting dayton vs USA smooth
dayton_39 = dayton_39.sort_values(by='date')
usa_sssm_comb= usa_sssm_comb.sort_values(by='date')
plt.plot(dayton_39['date'], dayton_39['value'], color='b',label='Dayton')
plt.plot(usa_sssm_comb['date'], usa_sssm_comb['value'], color='r', label='USA')
plt.title('Dayton vs USA median price (Smooth data)')
plt.legend()
plt.show()

In [ ]:
# plotting dayton vs USA smooth
dayton_39 = dayton_39.sort_values(by='date')
usa_sssm_comb= usa_sssm_comb.sort_values(by='date')
plt.plot(dayton_39['date'], dayton_39['value'], color='b',label='Dayton')
plt.plot(usa_sssm_comb['date'], usa_sssm_comb['value'], color='r', label='USA')
plt.title('Dayton vs USA median price (Smooth data)')
plt.legend()
plt.show()

## Finding on Smooth vs Raw data sets  
- The raw data has more spikes compared to the smooth data (thus the name 'smooth')
- As expected the correlation also dropped slightly (0.95 for smooth to 0.93 for raw)

In [ ]:
# wrangling data for scatter plot

# getting the listing for Dayton, OH
dayton_inventory = issm[issm['region_id'] == 394521]
len(dayton_inventory.sort_values(by='date'))

In [ ]:
# getting the listing (raw) for Dayton, OH
dayton_inventory_r = irsm[irsm['region_id'] == 394521]
len(dayton_inventory_r.sort_values(by='date'))

In [ ]:
# combining price and listing
dayton_sssm_issm = pd.merge(dayton_39, dayton_inventory, on='date',suffixes=['_price','_listing'])
len(dayton_sssm_issm)

In [ ]:
# combining price and listing (raw)
dayton_srsm_irsm = pd.merge(dayton_39r, dayton_inventory_r, on='date',suffixes=['_price','_listing'])
len(dayton_srsm_irsm)

In [ ]:
# extract columns for scatter plot
dayton_sssm_issm_sc_df= dayton_sssm_issm[['date','value_price','value_listing']]
dayton_sssm_issm_sc_df.head()

In [ ]:
# extract columns for scatter plot
dayton_srsm_irsm_sc_df= dayton_srsm_irsm[['date','value_price','value_listing']]
dayton_srsm_irsm_sc_df.head()

In [ ]:
# get the linear regression parameters smooth data
(slope, intercept, rvalue, pval, stderr) = stats.linregress(dayton_sssm_issm_sc_df['value_listing'],\
                                                            dayton_sssm_issm_sc_df['value_price'])
rvalue

In [ ]:
# plot the scatter plot and regression line with smooth data
dayton_sssm_issm_sc_df.plot(kind='scatter',x='value_listing', y='value_price', title='house price vs the number of listing (smooth data)',\
                            ylabel='median house price', xlabel = 'number of listing')
regr_yval = dayton_sssm_issm_sc_df['value_listing']*slope + intercept
plt.plot(dayton_sssm_issm_sc_df['value_listing'], regr_yval, color='r')
print(f'r squared value is {rvalue**2:.5f}')

In [ ]:
# linear regression for raw
(slope_r, intercept_r, rvalue_r, pval_r, stderr_r) = stats.linregress(dayton_srsm_irsm_sc_df['value_listing'],\
                                                            dayton_srsm_irsm_sc_df['value_price'])
rvalue_r

In [ ]:
# plot the scatter plot and regression line for raw
dayton_srsm_irsm_sc_df.plot(kind='scatter',x='value_listing', y='value_price', title='house price vs the number of listing (raw)',\
                           ylabel='median house price', xlabel='number of listing')
regr_yval_r = dayton_srsm_irsm_sc_df['value_listing']*slope_r + intercept_r
plt.plot(dayton_srsm_irsm_sc_df['value_listing'], regr_yval_r, color='r')
print(f'r squared value is {rvalue_r**2:.5f}')

## Finding on linear regression for smooth vs raw
- From the graph we can we that there is generally a negative correlation between the house price and number of listing. This makes sense from a supply-demand perspective. As the more supply (listing) there is, the lower the price will be given the same demand. The r-squared value is significant, meaning the regression model is quite reliable.  
    - Comparing the smooth to raw, the smooth data has higher r-squred value (0.43 vs 0.26), meaning the linear regression model using smoothed data is more predictable, because the data seems to have been 'compacted' during the smoothing process as we can see the data points are closer together to the regression line in the smooth dataset.


In [ ]:
# Display the location of New York, Killeen and Davenport on a map, with median price as size of the dot
# NY 40.7128° N, 74.0060° W (40.7128, -74.0060)
# Killeen, TX 31.1171° N, 97.7278° W (31.1171, -97.7278)
# Davenport, IA 41.5236° N, 90.5776° W (41.5236, -90.5776)
data ={'City': ['New York, NY', "Killeen, TX", "Davenport, IA"],
      'Lat': [40.7128, 31.1171, 41.5236 ],
      'Lng': [-74.0060, -97.7278, -90.5776],
      'Price':[ ny_r.loc[min(ny_r.index), 'value'], killeen_r.loc[min(killeen_r.index), 'value'], \
               davenport_r.loc[min(davenport_r.index), 'value'] ]
               }
city_df = pd.DataFrame(data)
city_df


In [ ]:
%%capture --no-display
usa_extent = (-130, 24, -64, 50)
map_pl = city_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", size="Price", scale=0.05, color= "City", frame_height=550, \
                              frame_width=750, #x_range=usa_extent[0:2], y_range=usa_extent[2:]
                              xlim=(-127, -64), ylim=(35,40)
                              )
map_pl

# Explanation  
  The locations of the cities that we have chosen as the representation of the 3 city sizes (big, medium, small). The dot size indicates the relative median house price in each city.

In [ ]:
# plotting big city (NY), Medium city (Killeen, TX), small city (Davenport) vs USA for raw
# source: https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

#show background for before and after covid in colors.
ny_r = ny_r.sort_values(by='date')
killeen_r =killeen_r.sort_values(by='date')
usa_srsm_comb = usa_srsm_comb.sort_values(by='date')

fig, ax = plt.subplots()
highlight_date = pd.to_datetime('2020-01-01')

ax.plot(ny_r['date'], ny_r['value'], color='b',label='New York, NY')
ax.plot(killeen_r['date'], killeen_r['value'], color='g',label='Killeen, TX')
ax.plot(davenport_r['date'], davenport_r['value'], color='y',label='Davenport, IA')
ax.plot(usa_srsm_comb['date'], usa_srsm_comb['value'], color='r', label='USA')

# highlight background before covid
ax.axvspan(np.min(ny_r['date']), highlight_date, alpha=0.2, color='lightblue')
# highlight background after covid
ax.axvspan(highlight_date, np.max(ny_r['date']), alpha=0.2, color='orange')

ax.set_title('Different-sized cities (based on Population) vs USA median house price (raw data)')
ax.set_ylabel('median house price')
ax.legend()
ax.grid(True)
plt.show()

## Finding on different sized cities compared to national house price trend
- As expected the big city has the highest price, followed by medium and finally the small city.
- One would expect the big city to have a higher correlation with national price than the smaller ones, because they have a bigger weight when compared to the smaller cities in the calculation of the national price, but it turned out that Killeen (a medium city) has the highest correlation (0.982) vs big city New York's 0.957 and small city Davenport's 0.911. This could be because the national price is a median price which is closest to the medium sized city's price (50th percentile price). As it can be reasonably assumed that when ranked by price, the bigger the city the higher its median price.
- House prices dropped across the country in 2008 because of the Great Financial Crisis (GFC), during which the big city price was impacted the most, while the smaller cities were affected the least. It took NY ten years (2018) to recover the drop in price during the GFC.
- Similar to Australia, the house prices in the US continue to price even more markedly during the pandemic (orange part of the graph)


In [ ]:
# calculating correlations with national price
us_dayton = pd.merge( dayton_39[['date','value']], usa_sssm_comb[['date', 'value']], on='date',suffixes=['_dayton', '_us'])
us_dayton_corr = us_dayton['value_dayton'].corr(us_dayton['value_us'])
us_dayton_corr


In [ ]:
us_dayton_r = pd.merge( dayton_39r[['date','value']], usa_srsm_comb[['date', 'value']], on='date',suffixes=['_dayton', '_us'])
us_dayton_corr_r = us_dayton_r['value_dayton'].corr(us_dayton_r['value_us'])
us_dayton_corr_r

In [ ]:
us_ny_r = pd.merge( ny_r[['date','value']], usa_srsm_comb[['date', 'value']], on='date',suffixes=['_ny', '_us'])
us_ny_corr_r = us_ny_r['value_ny'].corr(us_ny_r['value_us'])
us_ny_corr_r

In [ ]:
us_ki_r = pd.merge( killeen_r[['date','value']], usa_srsm_comb[['date', 'value']], on='date',suffixes=['_ki', '_us'])
us_ki_corr_r = us_ki_r['value_ki'].corr(us_ki_r['value_us'])
us_ki_corr_r

In [ ]:
us_dp_r = pd.merge( davenport_r[['date','value']], usa_srsm_comb[['date', 'value']], on='date',suffixes=['_dp', '_us'])
us_dp_corr_r = us_dp_r['value_dp'].corr(us_dp_r['value_us'])
us_dp_corr_r

In [ ]:
# bar chart for correlation
data=[us_ny_corr_r, us_ki_corr_r, us_dp_corr_r]
labels= ['New York (big)', 'Killeeen (medium)', 'Davenport (small)']
plt.bar(labels, data)
plt.ylim(.9, 1)
plt.title("correlations of city price with national median price")
plt.ylabel('Correlation value')
for i, v in enumerate(data):
    plt.text(i, v, str(f"{v:.3f}"), ha='center', va='bottom')
plt.show()

# Covid time analysis

In [ ]:
dayton_covidtime = dayton[(dayton['date'] >= '1-1-2019') & (dayton['date']<='31-12-2023')]
dayton_covidtime

In [ ]:
# creating a new column to identify if the date is before or during covid
dayton_covidtime = dayton_covidtime.loc[ dayton_covidtime['region_id'] ==394521, :]
# dayton_covidtime = dayton_covidtime[ dayton_covidtime['region_id'] ==394521]
dayton_covidtime['covid']='during'
dayton_covidtime.head()

In [ ]:
dayton_b4covid = dayton[(dayton['date'] >= '1-1-2014') & (dayton['date']<='31-12-2018')]
dayton_b4covid.head()

In [ ]:
dayton_b4covid = dayton_b4covid[dayton_b4covid['region_id']==394521]
# creating a new column to identify if the date is before or during covid
dayton_b4covid['covid'] = "before"
dayton_b4covid.head()

In [ ]:
# plot the before and after covid dayton side by side
# concat the data vertically.
dayton_b4_during_covid= pd.concat([dayton_b4covid[['date','value','covid']], dayton_covidtime[['date','value','covid']]],axis=0)
dayton_b4_during_covid

In [ ]:
# using seaborn to plot a line graph with different colors for before and during covid.
sns.lineplot(data = dayton_b4_during_covid, x='date', y='value', hue='covid')
plt.legend(title='covid')
plt.show()

## Analysis of house prices before and during covid  
  - It appears that although the pandemic does not affect the cycle of the house price much (still around 1 year), it does amaplify the magnitude of the cycle (there is a bigger difference between the trough and crest of the cycle).  
  - It also appears during covid the price seems less stable as there is more volatility as seen from multiple mini-cycles towards the beginning of 2022.

In [ ]:
sssm_subset = sssm_combine.sample(10000)
sssm_subset